# This is to have a look at the freely moving movement statistics

In [ ]:
from pathlib import Path
import cottage_analysis.io_module.onix as onix
import cottage_analysis.io_module.harp as harp
import cottage_analysis.ephys.preprocessing as prp
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd 
from scipy import stats
import cv2
import znamcalib.calibrate_lighthouse as light

In [ ]:
processed_path = Path('/camp/lab/znamenskiyp/home/shared/projects/blota_onix_pilote/')
data_path = Path('/camp/lab/znamenskiyp/data/instruments/raw_data/projects/blota_onix_pilote/')
mouse = 'BRAC7448.2d'
session = 'S20230412'
session_path = data_path / mouse / session

processed_dir = processed_path/mouse/session
processed_dir.mkdir(exist_ok=True)

#We should be able to programatically do this. 

ephys = 'R163257'
stimulus = 'R154624_SpheresPermTubeReward'
headfixed_camera = 'R161841_BRAC7448.2d'
fm_camera =  'R162624_BRAC7448.2d'

ephys_path = session_path / ephys
stimulus_path = session_path / stimulus
headfixed_cam_path = session_path / headfixed_camera
fm_cam_path = session_path / fm_camera

### Loading the data

In [ ]:
processed_photodiode = onix.load_ts4231(ephys_path)
processed_photodiode

The numbers I divide them by is how many LSBs (least significant bits) equal one SI unit. 

In [ ]:
processed_bno = onix.load_bno055(ephys_path)
processed_bno.keys()

quaternion = processed_bno['quaternion']
#quaternion = quaternion/2**14 but the rest make sense (???)
quaternion = quaternion
linear = processed_bno['linear']
linear = linear 
euler = processed_bno['euler']
euler = euler
gravity = processed_bno['gravity']
gravity = gravity 

In [ ]:
def plot_bno(start, end):

    fig, axs = plt.subplots(2, 2, figsize=(9, 9))

    fig.suptitle(f'BNO055 (timestamps {start}-{end})')

    timeslice=range(start, end)

    axs[0, 0].plot(quaternion[timeslice, 3])
    axs[0, 0].plot(quaternion[timeslice, 0])
    axs[0, 0].plot(quaternion[timeslice, 1])
    axs[0, 0].plot(quaternion[timeslice, 2])
    axs[0, 0].set_title('Quaternion')

    axs[0, 1].plot(linear[timeslice, 0])
    axs[0, 1].plot(linear[timeslice, 1])
    axs[0, 1].plot(linear[timeslice, 2])
    axs[0, 1].set_title('Linear acceleration')

    axs[1, 1].plot(euler[timeslice, 0])
    axs[1, 1].plot(euler[timeslice, 1])
    axs[1, 1].plot(euler[timeslice, 2])
    axs[1, 1].set_title('Euler Vector')

    axs[1, 0].plot(gravity[timeslice, 0])
    axs[1, 0].plot(gravity[timeslice, 1])
    axs[1, 0].plot(gravity[timeslice, 2])
    axs[1, 0].set_title('Gravity vector')

plot_bno(150000, 150400)

### Acceleration statistics

The linear acceleration output is an array of time*component(x, y, z)

I need to plot these numbers together with the movement of the animal to see 

1. What are components 1, 2, and 3
2. Does it make any sense at all?

Alternatively, I can stand on the freely-moving arena with the headstage on my hand, and move it along the three planes that I'd suspect would be useful. Also use that chance to record three proper calibration images. This is probably the quickest way forward. 

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(9, 12))
fig.suptitle('Linear accelerations distribution')


axs[0].hist(linear[:,0])
axs[0].set_xlabel('Acceleration in x (m/s^2)')
axs[1].hist(linear[:,1])
axs[1].set_xlabel('Acceleration in y (m/s^2)')
axs[2].hist(linear[:,2])
axs[2].set_xlabel('Acceleration in z (m/s^2)')


### Decomposing heading

We will try to decompose yaw, pitch and roll from the euler vector. Then, we will use that information to track which components of linear acceleration change when each of these change.

Are the three Euler angles yaw, pitch and roll? As per the documentation of BNO055, it's heading-roll-pitch. 

If this was true, heading should span a whole circle, so from 0 to 2pi. This is weird: it does look reasonable, but not when you look at the numbers. This is true neither in radians nor in degrees. 

Ah, no. I divided it by 16 and 900, but apparently have to divide it by nothing!

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(9, 12))
fig.suptitle('Euler angles distribution')


axs[0].hist(euler[:,0])
axs[0].set_xlabel('Heading in degrees')
axs[1].hist(euler[:,1])
axs[1].set_xlabel('Roll in degrees')
axs[2].hist(euler[:,2])
axs[2].set_xlabel('Pitch in degrees')

### Can I have a meaningful look at the cameras?

Let's try

In [ ]:
camera_dir = session_path / fm_camera
acquisition= 'freely_moving'

camera_metadata = onix.load_camera_times(camera_dir, acquisition)
print(camera_metadata.keys())

processed_breakout = onix.load_breakout(ephys_path)
dio = processed_breakout['dio']
print(dio.keys())


filtered_dio = dict()
filtered_dio['Clock'], filtered_dio['DI0'] = prp.clean_di_channel(dio['Clock'], dio['DI0'])
print(filtered_dio['DI0'])
print(filtered_dio['Clock'])

cam2_metadata = camera_metadata['cam2_camera']
print(cam2_metadata.keys())
camera_frames = len(cam2_metadata['frame_id'])
breakout_frames = (len(filtered_dio['DI0']))/2

print(f'cam2 has {camera_frames} frames')
print(f'The breakout board accounts for {breakout_frames} frames')

def cam2onix(filtered_dio, cam2_metadata):

    onix_first = filtered_dio['Clock'][filtered_dio['DI0']==True][0]
    onix_last = filtered_dio['Clock'][filtered_dio['DI0']==True][-1]
    camera_first = cam2_metadata['camera_timestamp'].iloc[0]
    camera_last = cam2_metadata['camera_timestamp'].iloc[-1] 

    #y=intercept+slope*x

    intercept = onix_first
    slope = (onix_last-onix_first)/(camera_last-camera_first)

    onix_timestamp = intercept+slope*(cam2_metadata['camera_timestamp']-camera_first)

    return onix_timestamp

cam2_metadata['onix_timestamp'] = cam2onix(filtered_dio, cam2_metadata)
cam2_metadata['onix_timestamp']

plt.plot(filtered_dio['Clock'][filtered_dio['DI0']==True], color = 'blue')
plt.plot(cam2_metadata['onix_timestamp'], color = 'red')

### Identify a section of the recording where the animal is moving in a particularly interesting way. 

For example, in this section, one of the components of the euler vector increases monotonically

In [ ]:
plot_bno(113050, 113320)


In [ ]:
def find_nearest(Array, x):
    dif_Array = np.absolute(Array-x) # use of absolute() function to find the difference 
    index = dif_Array.argmin() # find the index of minimum difference element
    return Array[index]

start = processed_bno['onix_time'][4430]
end = processed_bno['onix_time'][4700]

start_frame_onix = find_nearest(cam2_metadata['onix_timestamp'], start)
start_frame = cam2_metadata[cam2_metadata['onix_timestamp']==start_frame_onix]
end_frame_onix = find_nearest(cam2_metadata['onix_timestamp'], end)
end_frame = cam2_metadata[cam2_metadata['onix_timestamp']==end_frame_onix]



And we show beggining and end!

In [ ]:
def bno_start_end(video_path, start_frame, end_frame):
    video = cv2.VideoCapture(video_path)
    video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    ret, frame_start = video.read()
    video.set(cv2.CAP_PROP_POS_FRAMES, end_frame)
    ret, frame_end = video.read()
    fig, axs = plt.subplots(2, 1, figsize=(50, 50))
    fig.suptitle(f'Frames {start_frame}-{end_frame}', size = 40)
    axs[0].imshow(frame_start)
    axs[1].imshow(frame_end)

video_path = '/camp/lab/znamenskiyp/data/instruments/raw_data/projects/blota_onix_pilote/BRAC7448.2d/S20230412/R162624_BRAC7448.2d/cam2_camera/cam2_camera_2023-04-12T16_26_24.mp4'
#video_path = '/camp/lab/znamenskiyp/data/instruments/raw_data/projects/blota_onix_pilote/BRAC7448.2d/S20230412/R162624_BRAC7448.2d/cam1_camera/cam1_camera_2023-04-12T16_26_24.mp4'


bno_start_end(video_path, start_frame.index[0], end_frame.index[0])

### Find Lighthouse and check

In [ ]:
processed_photodiode = onix.load_ts4231(ephys_path)
diode3 = processed_photodiode[3]


In [ ]:
calibration = Path('/camp/lab/znamenskiyp/data/instruments/raw_data/projects/blota_onix_calibration/lighthouse_calibration/S20230412')
transform_matrix = light.calibrate_session(calibration, 20)
print(transform_matrix)

In [ ]:
trans_data = light.transform_data(diode3, transform_matrix)

In [ ]:
light.plot_single_occupancy(trans_data)

And we find the adequate two points

In [ ]:
trans_data

In [ ]:
#The onix timestamp closest to start and end

lighthouse_start = find_nearest(trans_data['clock'], start)
lighthouse_start_frame = trans_data[trans_data['clock']==lighthouse_start]
lighthouse_end = find_nearest(trans_data['clock'], end)
lighthouse_end_frame = trans_data[trans_data['clock']==lighthouse_end]

In [ ]:
def lighthouse_start_end(start, end):
    lighthouse_start = find_nearest(trans_data['clock'], start)
    lighthouse_start_frame = trans_data[trans_data['clock']==lighthouse_start]
    lighthouse_end = find_nearest(trans_data['clock'], end)
    lighthouse_end_frame = trans_data[trans_data['clock']==lighthouse_end]
    fig, axs = plt.subplots(1, 1, figsize=(9, 9))
    fig.suptitle(f'Lighthouse {start}-{end}', size = 30)
    axs.set_ylim(0, 10)
    axs.set_xlim(-10, 70)
    axs.plot(lighthouse_start_frame['x'], lighthouse_start_frame['z'], 'bo')
    axs.plot(lighthouse_end_frame['x'], lighthouse_end_frame['z'], 'ro')
    #axs.plot(55, 55, '*', markersize = 30)

lighthouse_start_end(start, end)

### Get stable north

In [ ]:
euler[:,0]

In [ ]:
north = np.where((1>euler[:,0]))
north = north[0]
len(north)

In [ ]:
start = processed_bno['onix_time'][north[200]]
end = processed_bno['onix_time'][north[210]]

start_frame_onix = find_nearest(cam2_metadata['onix_timestamp'], start)
start_frame = cam2_metadata[cam2_metadata['onix_timestamp']==start_frame_onix]
end_frame_onix = find_nearest(cam2_metadata['onix_timestamp'], end)
end_frame = cam2_metadata[cam2_metadata['onix_timestamp']==end_frame_onix]

bno_start_end(video_path, start_frame.index[0], end_frame.index[0])

In [ ]:
start = processed_bno['onix_time'][north[200]]
end = processed_bno['onix_time'][north[210]]

lighthouse_start_end(start, end)